In [13]:
# Versión final

# Aproximación de Simpson compuesta para integral 1D
def trapz1d (n, f, a, b):
    integ = (f(a)+f(b))/2
    h = (b-a)/n
    for i in range(1, n):
        x = a + i*h
        integ += f(x)
    integ *= h
    return integ


# Aproximación de Simpson compuesta para integral 2D
def trapz2d(f, a, b, c, d, n, m):
    integ = 0
    h = (b-a)/n

    # Veamos si c, d son constantes (para áreas de integración rectangulres) 
    # y las convertimos a funciones para que el resto del código funcione
    phi1 = c if callable(c) else lambda x: c
    phi2 = d if callable(d) else lambda x: d

    # En vez de este pedazo, podríamos introducir un if dentro del ciclo y cambiar el coeficiente
    integ = (trapz1d(m, lambda y: f(a, y), phi1(a), phi2(a)) + trapz1d(m, lambda y: f(b, y), phi1(b), phi2(b)))/2
    
    for i in range(1,n-1):
        x = a + i*h
        g = lambda y: f(x,y)
        integ += trapz1d(m, g, phi1(x), phi2(x))
    integ *= h
    return integ


In [14]:
# Versión final

# Aproximación de Simpson compuesta para integral 1D
def simps1d(n, f, a, b):
    integ = f(a)+f(b)   # Valor inicial: primera parte de la suma
    h = (b-a)/n

    # Valores iniciales para el ciclo
    segSum = 0
    terSum = 0
    predX = a # xi anterior

    for i in range(1, n):
        x = a + h*i
        segSum += f(x)
        terSum += f((predX + x)/2)
        predX = x  # Valor para el siguiente ciclo

    # Valores faltantes del ciclo
    terSum += f((predX + b)/2)

    # Coeficientes de las sumas
    segSum *= 2
    terSum *= 4

    # Agregamos valores a aprox y multiplicamos por factor
    integ += segSum + terSum
    integ *= h/6

    return integ


# Aproximación de Simpson compuesta para integral 2D
def simps2d(f, a, b, c, d, n, m):
    # Veamos si c, d son constantes (para áreas de integración rectangulres) 
    # y las convertimos a funciones para que el resto del código funcione
    phi1 = c if callable(c) else lambda x: c
    phi2 = d if callable(d) else lambda x: d 

    # Integral interior como función de x
    F = lambda x: simps1d(m, lambda y: f(x, y), phi1(x), phi2(x))

    integ = F(a)+F(b)   # Valor inicial: primera parte de la suma
    h = (b-a)/n

    # Valores iniciales para el ciclo
    segSum = 0
    terSum = 0
    predX = a
    for i in range(1, n):
        x = a + i*h
        segSum += F(x)
        terSum += F((predX+x)/2)
        predX = x # Valor para el siguiente ciclo

    # Valores faltantes del ciclo
    terSum += F((predX+b)/2)

    # Coeficientes de las sumas
    segSum *= 2
    terSum *= 4

    # Agregamos valores a aprox y multiplicamos por factor
    integ += segSum + terSum
    integ *= h/6

    return integ

In [15]:
# Versión final

# Aproximación de Gauss compuesta para integral 1D
# k es número de abscisas
def gauss1d(f, a, b, n, k):
    h = (b-a)/n
    x, w  = np.polynomial.legendre.leggauss(k)  # Pesos y abscisas
    integ = 0.0

    # Ciclo que itera intervalos
    for i in range(n):
        xa = a + i*h
        xb = xa + h
        T = lambda u: (xa + xb + u*h)/2     # Transformación para intervalo [-1,1]
        # Integración gaussiana para cada intervalo
        for l in range(k):
            integ += w[l] * f(T( x[l] ))
    integ *= h/2
    return integ

def gauss2d(f, a, b, c, d, n, m, kn, km):
    # Veamos si c, d son constantes (para áreas de integración rectangulres) 
    # y las convertimos a funciones para que el resto del código funcione
    phi1 = c if callable(c) else lambda x: c
    phi2 = d if callable(d) else lambda x: d 

    F = lambda x: gauss1d(lambda y: f(x,y), phi1(x), phi2(x), m, km)

    h = (b-a)/n
    x, w = np.polynomial.legendre.leggauss(kn)
    integ = 0.0

    # Ciclo que itera intervalos
    for i in range(n):
        xa = a + h*i
        xb = xa + h
        T = lambda u: (xa + xb + u*h)/2 # Transformación para intervalo [-1,1]
        # Integración gaussiana para cada intervalo
        for l in range(kn):
            integ += w[l] * F(T( x[l] ))
    integ *= h/2
    return integ


In [17]:
import numpy as np

# Definimos la función a integrar
def f(x, y):
    return np.sin(x*y)

# Definimos los límites de integración para la variable x
a = 0.0
b = np.pi

# Definimos el número de subintervalos en cada dirección
n = 10
m = 10

# Definimos las funciones phi1 y phi2 como funciones que dependen de x
def phi1(x):
    return x/2

def phi2(x):
    return 3*x/2

  
# Valor real de la doble integral (según WolframAlpha)
valorReal = 0.4249603946480658226404103877736619464629130316183373589717236607

In [20]:
print("="*60+"\n")
print("Aproximación de la integral en dos dimensiones")
print("Valor real: ", valorReal)
print("")

minExp = 0
maxExp = 10
abscisas = [2,4]

for i in range(minExp, maxExp+1):
    n = 2**i
    m = 2**i
    integral_trapz2d = trapz2d(f, a, b, phi1, phi2, n, m)
    errorTrapz2d = abs(valorReal-integral_trapz2d)
    integral_simp2d = simps2d(f, a, b, phi1, phi2, n, m)
    errorSimps2d = abs(valorReal-integral_simp2d)
    print("n="+str(n)+", m="+str(m))
    print("Trapecio:             ", integral_trapz2d, " ", "error abs: ", errorTrapz2d)
    print("Simpson:              ", integral_simp2d, " ", "error abs: ", errorSimps2d)
    for numAbs in abscisas:
      integral_gauss2d = gauss2d(f, a, b, phi1, phi2, n, m, numAbs, numAbs)
      errorGauss2d = abs(valorReal-integral_gauss2d)
      print("Gauss con "+ str(numAbs) +" abscisas: ", integral_gauss2d, " ", "error abs: ", errorGauss2d)
    print("")
print("="*60)


Aproximación de la integral en dos dimensiones
Valor real:  0.42496039464806584

n=1, m=1
Trapecio:              -0.46839948850405455   error abs:  0.8933598831521203
Simpson:               1.0716706382439358   error abs:  0.64671024359587
Gauss con 2 abscisas:  0.5531789177744291   error abs:  0.12821852312636323
Gauss con 4 abscisas:  -0.12464170777021595   error abs:  0.5496021024182818

n=2, m=2
Trapecio:              -0.6479627202632902   error abs:  1.072923114911356
Simpson:               0.7556616291111019   error abs:  0.33070123446303606
Gauss con 2 abscisas:  0.17844862858525984   error abs:  0.246511766062806
Gauss con 4 abscisas:  0.4232943363002707   error abs:  0.0016660583477951518

n=4, m=4
Trapecio:              0.9662010459661883   error abs:  0.5412406513181225
Simpson:               0.3415654663000797   error abs:  0.08339492834798612
Gauss con 2 abscisas:  0.4832063051372708   error abs:  0.058245910489204966
Gauss con 4 abscisas:  0.42481226043441683   error abs